In [5]:
from yelpapi import YelpAPI
import argparse
from pprint import pprint
from api_key import api_key
import pickle

In [2]:
API_KEY = api_key
yelp_api = YelpAPI(API_KEY)
response = yelp_api.search_query(term='coffee', location='chicago, IL', sort_by='rating', limit=50, offset=51)
pprint(response)





{'businesses': [{'alias': 'brewpoint-coffee-elmhurst',
                 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}],
                 'coordinates': {'latitude': 41.89922, 'longitude': -87.94134},
                 'display_phone': '(224) 542-8263',
                 'distance': 21818.0477763561,
                 'id': 'u23RsNsjyRbwWqGaZ2ZZ2Q',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/DGflZvUSUJTXWoNe7y62ng/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '124 W Park Ave',
                              'address2': None,
                              'address3': '',
                              'city': 'Elmhurst',
                              'country': 'US',
                              'display_address': ['124 W Park Ave',
                                                  'Elmhurst, IL 60126'],
                              'state': 'IL',
                              'zip_code': '60126'},
          

                 'categories': [{'alias': 'breakfast_brunch',
                                 'title': 'Breakfast & Brunch'},
                                {'alias': 'coffee', 'title': 'Coffee & Tea'},
                                {'alias': 'tradamerican',
                                 'title': 'American (Traditional)'}],
                 'coordinates': {'latitude': 41.7805013,
                                 'longitude': -87.5962316},
                 'display_phone': '(773) 891-4240',
                 'distance': 15419.237757080156,
                 'id': '-Tp77Ce5OCpP1-KXUYtX9A',
                 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/ayXC_8IHewJgZ-sKVZTYRw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '6300 S Woodlawn',
                              'address2': '',
                              'address3': '',
                              'city': 'Chicago',
                              'country': 'US',
            

In [3]:
%load_ext autoreload
%autoreload 1
%aimport yelp_to_dict
from yelp_to_dict import add_to_shop_dict


In [4]:
coffee_shop_dict={}
API_KEY = api_key
yelp_api = YelpAPI(API_KEY)
for next_50 in range(0,950, 51):
    response = yelp_api.search_query(term='coffee shop', location='chicago, IL', sort_by='rating', limit=50, offset=next_50)
    add_to_shop_dict(response, coffee_shop_dict)


black-fodder-coffee-chicago-9
did not work
the-rolling-bean-chicago-3
did not work
jericho-coffee-chicago
did not work
nefertiti-gourmet-coffee-chicago
did not work
pierogi-wagon-chicago
did not work


In [10]:
coffee_shop_dict

{'f5p7scO1Q9UdaGbYe9bjFg': ('two-hearted-queen-chicago',
  False,
  41.9433099,
  -87.65928,
  '1201 W Roscoe St, Chicago, IL 60657',
  '1201 W Roscoe St',
  '60657',
  '$',
  5.0,
  239),
 'gxUFFsSLa-IyouuRD2Ma6w': ('spinning-j-chicago-2',
  False,
  41.899391,
  -87.696991,
  '1000 N California Ave, Chicago, IL 60622',
  '1000 N California Ave',
  '60622',
  '$$',
  4.5,
  314),
 'VScrypD_qyg_Jzbz59BXmg': ('backlot-coffee-chicago',
  False,
  41.9533408903263,
  -87.7319341152907,
  '3982 N Avondale Ave, Chicago, IL 60641',
  '3982 N Avondale Ave',
  '60641',
  '$$',
  5.0,
  73),
 'qYTLqAtRvJ-RczO40sAFvg': ('sip-of-hope-chicago-2',
  False,
  41.92444,
  -87.70389,
  '3039 W Fullerton Ave, Chicago, IL 60647',
  '3039 W Fullerton Ave',
  '60647',
  '$',
  5.0,
  57),
 'G-LUoHk_E_XCjSoSsaflFw': ('sawada-coffee-chicago',
  False,
  41.8836669921875,
  -87.6489334106445,
  '112 N Green St, Chicago, IL 60607',
  '112 N Green St',
  '60607',
  '$$',
  4.5,
  527),
 'CZAVqR6lI50rVbN7E78aHA

In [6]:
with open('yelp_coffee_shop_dict.pk', 'wb') as write_file:
    pickle.dump(coffee_shop_dict, write_file)

## Postgres

In [73]:
import psycopg2

In [74]:
conn = psycopg2.connect('dbname=coffee')

In [75]:
cur = conn.cursor()

In [76]:
for yelp_shop in response['businesses']:
    try:
        try:
            cur.execute("INSERT INTO coffeeshops (yelpid,yelalias,yelpisclosed,address_display,address1,addresszip,latitude,longitude,pricing,rating,reviewcount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (
                                                        yelp_shop['id'],
                                                        yelp_shop['alias'],
                                                        yelp_shop['is_closed'],
                                                        yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1],
                                                        yelp_shop['location']['address1'],
                                                        yelp_shop['location']['zip_code'],
                                                        yelp_shop['coordinates']['latitude'],
                                                        yelp_shop['coordinates']['longitude'],
                                                        yelp_shop['price'],
                                                        yelp_shop['rating'],
                                                        yelp_shop['review_count']))
        except:
            cur.execute("INSERT INTO coffeeshops (yelpid,yelalias,yelpisclosed,address_display,address1,addresszip,latitude,longitude,rating,reviewcount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (
                                                        yelp_shop['id'],
                                                        yelp_shop['alias'],
                                                        yelp_shop['is_closed'],
                                                        yelp_shop['location']['display_address'][0]+', ' + yelp_shop['location']['display_address'][1],
                                                        yelp_shop['location']['address1'],
                                                        yelp_shop['location']['zip_code'],
                                                        yelp_shop['coordinates']['latitude'],
                                                        yelp_shop['coordinates']['longitude'],
                                                        yelp_shop['rating'],
                                                        yelp_shop['review_count']))
    except:
        pass



In [77]:
# conn.rollback()

In [78]:
cur.execute("SELECT * FROM coffeeshops;")


InternalError: current transaction is aborted, commands ignored until end of transaction block


In [62]:
cur.fetchone()

('f5p7scO1Q9UdaGbYe9bjFg',
 'two-hearted-queen-chicago',
 False,
 '1201 W Roscoe St, Chicago, IL 60657',
 '1201 W Roscoe St',
 '60657',
 Decimal('41.9433099000000'),
 Decimal('-87.6592800000000'),
 '$',
 None,
 239,
 5)

In [63]:
conn.commit()

In [69]:
cur.close()

In [70]:
conn.close()